In [1]:
import pandas as pd
import numpy as np
import random
from scipy import stats
import seaborn as sns
#import scikitplot as skplt
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

np.random.seed(42)
random.seed(42)

In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.6 MB/s eta 0:00:00


In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.8 MB/s eta 0:00:00


In [4]:
import warnings
warnings.filterwarnings('ignore')

# EDA

In [5]:
train = pd.read_csv("train.csv")

numerical_cols = train.select_dtypes(include = ['int64', 'float64']).columns
categorical_cols = train.select_dtypes(include = ['object']).columns

train["price"] = np.log(train["price"])

train.accident = train.accident.fillna('None reported')

# train.fuel_type = train.fuel_type.fillna(train.fuel_type.mode(dropna=False)[0])
train.fuel_type = train.fuel_type.fillna('None reported')

# train.clean_title = train.clean_title.fillna(train.clean_title.mode(dropna=False)[0])
train.clean_title = train.clean_title.fillna('None reported')

def categorize_year(year):
    if 1992 <= year <= 1995:
        return 1
    elif year <= 1991:
        return -1
    elif 1996 <= year <= 2004:
        return 2
    elif 2006 <= year <= 2012:
        return 3
    elif year == 2005:
        return 4
    elif year == 2023:
        return 5
    else:
        return 0

train['year_category'] = train['model_year'].apply(categorize_year)

average_prices_brand = train.groupby('brand')['price'].mean().reset_index()
prem_car = ['Alfa',
 'Aston',
 'Audi',
 'BMW',
 'Bentley',
 'Bugatti',
 'Cadillac',
 'Chevrolet',
 'Ferrari',
 'Ford',
 'Genesis',
 'Jaguar',
 'Lamborghini',
 'Land',
 'Lucid',
 'Maserati',
 'Maybach',
 'McLaren',
 'Mercedes-Benz',
 'Polestar',
 'Porsche',
 'RAM',
 'Rivian',
 'Rolls-Royce',
 'Tesla']

def categorize_brand(brand):
    if brand in prem_car:
        return 1
    else:
        return 0

train['brand_category'] = train['brand'].apply(categorize_brand)

average_prices_ext_col = train.groupby('ext_col')['price'].mean().reset_index()
average_prices_model = train.groupby('model')['price'].mean().reset_index()

list(average_prices_model[average_prices_model['price'] > 100000]['model'])
prem_models = ['3500 Tradesman',
 '488 GTB Base',
 '500e Battery Electric',
 '570S Base',
 '570S Spider',
 '718 Cayman GTS',
 '720S Performance',
 '812 Superfast Base',
 '911 Carrera 4 GTS',
 '911 Carrera GTS',
 '911 GT2 RS',
 '911 GT3',
 '911 GT3 RS',
 '911 R',
 '911 Turbo',
 '911 Turbo Cabriolet',
 '911 Turbo S',
 'AMG E 53 4MATIC',
 'AMG GLS 63 4MATIC',
 'AMG GT AMG GT S',
 'Aventador LP700-4',
 'Aventador LP750-4 Superveloce',
 'Aventador S Base',
 'Aventador SVJ Base',
 'Bentayga Activity Edition',
 'Bentayga Azure First Edition',
 'Bentayga Onyx Edition',
 'Bentayga S',
 'Bentayga Speed',
 'Bentayga V8',
 'Bronco Raptor',
 'CT5-V Blackwing',
 'Carrera GT Base',
 'Cayman GTS',
 'Continental GT GT Speed',
 'Continental GT Speed',
 'Continental GT V8',
 'Continental GT V8 First Edition',
 'Continental GT V8 S',
 'Cullinan',
 'DeVille Base',
 'Escalade Sport',
 'F12berlinetta Base',
 'FF Base',
 'Flying Spur V8',
 'GT',
 'GTC4Lusso T',
 'Gallardo LP570-4 Superleggera',
 'Gallardo SE',
 'Ghost Base',
 'Grecale Modena',
 'Huracan EVO Base',
 'Huracan EVO Coupe',
 'Huracan LP580-2',
 'Huracan LP580-2S',
 'Huracan LP610-4',
 'Huracan Tecnica Coupe',
 'Levante GTS',
 'M8 Competition',
 'M8 Gran Coupe Competition',
 'MC20 Base',
 'Martin DB7 Vantage Volante',
 'Martin DBS Superleggera',
 'Martin DBX Base',
 'Maybach S 580 4MATIC',
 'Murcielago Base',
 'Phantom',
 'Phantom Drophead Coupe Drophead',
 'Pilot Elite',
 'R8 5.2 V10',
 'R8 5.2 V10 performance',
 'RS 7 4.0T',
 'RS 7 4.0T quattro',
 'RS Q8 4.0T',
 'RS Q8 4.0T quattro',
 'Roma Base',
 'Rover LR4 Lux',
 'Rover Range Rover P525 HSE SWB',
 'Rover Range Rover P530 SE',
 'Rover Range Rover P530 SE LWB 7 Seat',
 'Rover Range Rover P530 SE SWB',
 'S-Class S 580 4MATIC',
 'S4 Base',
 'SQ7 4.0T',
 'Shelby GT500 Base',
 'Sierra 1500 SLE1 Extended Cab',
 'Silverado 1500 Custom Trail Boss',
 'Taycan Turbo',
 'Urus Base',
 'Urus Pearl Capsule',
 'Veyron 16.4 Grand Sport',
 'Wraith Base']


def categorize_model(model):
    if model in prem_models:
        return 1
    else:
        return 0

train['model_category'] = train['model'].apply(categorize_model)

average_mileage_per_model = train.groupby('model')['milage'].mean()

# Добавляем новую фичу: Относительный пробег
train['relative_mileage'] = train.apply(
    lambda row: row['milage'] / average_mileage_per_model[row['model']], axis=1
)

# Опционально: Категоризируем относительный пробег
def mileage_category(relative_mileage):
    if relative_mileage < 0.8:
        return 'Low Mileage'
    elif relative_mileage < 1.2:
        return 'Average Mileage'
    else:
        return 'High Mileage'

train['mileage_category'] = train['relative_mileage'].apply(mileage_category)

train['brand_model'] = train['brand'] + '_' + train['model']
brand_counts = train['brand_model'].value_counts()

# Создаем две маски
high_frequency_brands = brand_counts[brand_counts > 500].index
low_frequency_brands = brand_counts[brand_counts <= 500].index

# Предварительно вычисляем среднюю цену для high_frequency_brands
high_frequency_means = train[train['brand_model'].isin(high_frequency_brands)] \
    .groupby('brand_model')['price'].mean()

# Предварительно вычисляем медианную цену для low_frequency_brands
low_frequency_median = train[train['brand_model'].isin(low_frequency_brands)]['price'].median()

# Создаем новую колонку с заполненными значениями
def get_filled_price(row):
    if row['brand_model'] in high_frequency_means:
        return high_frequency_means[row['brand_model']]
    return low_frequency_median

train['brand_model_price'] = train['brand_model'].map(high_frequency_means).fillna(low_frequency_median)

average_prices_year = train.groupby('model_year')['price'].mean().reset_index()

average_prices_brand.rename(columns={'price': 'average_prices_brand'}, inplace=True)
train_new = pd.merge(train, average_prices_brand, on='brand')
average_prices_year.rename(columns={'price': 'average_prices_year'}, inplace=True)
train_new = pd.merge(train_new, average_prices_year, on='model_year')

average_prices_model.rename(columns={'price': 'average_prices_model'}, inplace=True)
train_new = pd.merge(train_new, average_prices_model[['model',  'average_prices_model']], on='model')


numerical_cols = train_new.select_dtypes(include = ['int64', 'float64']).columns
categorical_cols = train_new.select_dtypes(include = ['object']).columns

numerical_cols = ['milage', 'relative_mileage', 'brand_model_price',
       'average_prices_brand', 'average_prices_year', 'average_prices_model']
cat_cols = ['brand', 'model', 'model_year', 'fuel_type', 'transmission',
       'ext_col', 'int_col', 'accident', 'clean_title', 'year_category',
       'brand_category', 'model_category', 'mileage_category', 'brand_model']
text_col = ['engine']
for i in cat_cols:
    train_new[i] = train_new[i].astype(str)

for i in text_col:
    train_new[i] = train_new[i].astype(str)

In [6]:
test = pd.read_csv("test.csv")

median_mileage = average_mileage_per_model.median()
test['brand_model'] = test['brand'] + '_' + test['model']
test['brand_model_price'] = test['brand_model'].map(high_frequency_means).fillna(low_frequency_median)
test['relative_mileage'] = test.apply(
    lambda row: row['milage'] / average_mileage_per_model.get(row['model'], median_mileage), axis=1
)
test['mileage_category'] = test['relative_mileage'].apply(mileage_category)


test['year_category'] = test['model_year'].apply(categorize_year)
test['model_category'] = test['model'].apply(categorize_model)
test['brand_category'] = test['brand'].apply(categorize_brand)
average_prices_dict = average_prices_model.set_index('model')['average_prices_model'].to_dict()
test_new = pd.merge(test, average_prices_brand, on='brand', how='left')
test_new = pd.merge(test_new, average_prices_year, on='model_year', how='left')
test_new['average_prices_model'] = test_new['model'].map(average_prices_dict)

numerical_cols = ['milage', 'relative_mileage', 'brand_model_price',
       'average_prices_brand', 'average_prices_year', 'average_prices_model']
cat_cols = ['brand', 'model', 'model_year', 'fuel_type', 'transmission',
       'ext_col', 'int_col', 'accident', 'clean_title', 'year_category',
       'brand_category', 'model_category', 'mileage_category', 'brand_model']
text_col = ['engine']
for i in cat_cols:
    test_new[i] = test_new[i].astype(str)

for i in text_col:
    test_new[i] = test_new[i].astype(str)

test_new["ids"] = test_new.index

In [31]:
all_feats = numerical_cols + cat_cols + text_col

X = train_new.drop(columns = ['price', 'id'])
y = train_new['price']
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42, train_size=0.8, shuffle=True)

data = pd.concat([X[all_feats],test_new[all_feats]])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])
# X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

X[numerical_cols] = data[numerical_cols].iloc[:len(X)]
test_new[numerical_cols] = data[numerical_cols].iloc[len(X)+1:]

## CatBoost

In [8]:
from catboost import CatBoostRegressor
import optuna
from optuna.samplers import TPESampler
from catboost import Pool
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import KFold

In [9]:
default_params = {
    'iterations':7000,
        'learning_rate':0.01,
        'loss_function':'RMSE',
        'use_best_model': True,
        'task_type':'GPU'
}

In [10]:
from catboost import CatBoostRegressor
from catboost import Pool

def fit_model(train_pool, validation_pool, params):
    model = CatBoostRegressor(
        **params,
        task_type='GPU'
    )

    return model.fit(
        train_pool,
        eval_set=validation_pool,
        verbose=1000,
    )

# Train old way


In [ ]:
train_pool = Pool(
    X_train, y_train,
    text_features = text_col,
    cat_features = cat_cols
)

validation_pool = Pool(
    X_test, y_test,
    text_features = text_col,
    cat_features= cat_cols
)

print('Train dataset shape: {}\n'.format(train_pool.shape))

model = fit_model(train_pool, validation_pool,defaul_params)

Train dataset shape: (28600, 17)

0:	learn: 0.8419351	test: 0.8417925	best: 0.8417925 (0)	total: 70.7ms	remaining: 8m 14s


KeyboardInterrupt: 

# Feature Selection

In [11]:
select_params = {
    'iterations':1000,
        'learning_rate':0.01,
        'loss_function':'RMSE',
        'use_best_model': True,
}

In [ ]:
def forward_selection(to_add_feats, params, eps=0.0):

    top_score = 10000

    features_dict = dict()
    features = []
    for i, add_feat in enumerate(to_add_feats):

        features.append(add_feat)
        features.sort()
        cat_features = [feat for feat in features if feat in cat_cols]
        text_features = [feat for feat in features if feat in text_col]

        if len(cat_features)>0:
            train_pool = Pool(
            X_train[features].values, y_train.values,
            text_features = text_features,
            feature_names = features,
            cat_features = cat_features
            )

            validation_pool = Pool(
                X_test[features].values, y_test.values,
                text_features = text_features,
                feature_names = features,
                cat_features= cat_features
            )
        else:
            train_pool = Pool(
            X_train[features].values, y_train.values,
            text_features = text_features,
            feature_names = features,
            )

            validation_pool = Pool(
                X_test[features].values, y_test.values,
                text_features = text_features,
                feature_names = features,
            )

        model = fit_model(train_pool,validation_pool,params)

        valid_score = model.best_score_['validation']['RMSE']
        features_dict[add_feat] = top_score - valid_score

        if valid_score < top_score - eps:
            top_score = valid_score
        else:
            features.remove(add_feat)
            features.sort()

    return features, features_dict

In [ ]:
feats_prev = numerical_cols+cat_cols+text_col

In [ ]:
FEATS_forw, features_dict_03 = forward_selection(feats_prev, select_params,eps=0.0)

0:	learn: 0.8394738	test: 0.8419702	best: 0.8419702 (0)	total: 5.88ms	remaining: 5.87s
999:	learn: 0.5850332	test: 0.5911981	best: 0.5911981 (999)	total: 3.59s	remaining: 0us
bestTest = 0.5911980898
bestIteration = 999
0:	learn: 0.8393972	test: 0.8419010	best: 0.8419010 (0)	total: 4.24ms	remaining: 4.24s
999:	learn: 0.5587816	test: 0.5660383	best: 0.5660383 (999)	total: 4.89s	remaining: 0us
bestTest = 0.5660382835
bestIteration = 999
0:	learn: 0.8393972	test: 0.8419010	best: 0.8419010 (0)	total: 20.2ms	remaining: 20.2s
999:	learn: 0.5538550	test: 0.5612538	best: 0.5612538 (999)	total: 3.53s	remaining: 0us
bestTest = 0.561253755
bestIteration = 999
0:	learn: 0.8394111	test: 0.8419169	best: 0.8419169 (0)	total: 4.75ms	remaining: 4.75s
999:	learn: 0.5462945	test: 0.5545627	best: 0.5545627 (999)	total: 3.42s	remaining: 0us
bestTest = 0.5545626659
bestIteration = 999
0:	learn: 0.8389605	test: 0.8414628	best: 0.8414628 (0)	total: 5.26ms	remaining: 5.25s
999:	learn: 0.5147370	test: 0.5213346	

In [ ]:
features_dict_03

{'milage': 9999.40880191018,
 'relative_mileage': 0.025159806344015267,
 'brand_model_price': 0.0047845284924213916,
 'average_prices_brand': 0.006691089120195337,
 'average_prices_year': 0.03322810846152591,
 'average_prices_model': 0.013658647217849595,
 'brand': 4.081304053427104e-05,
 'model': 0.001222238512699425,
 'model_year': 0.0001975213664660025,
 'fuel_type': 0.00037448677792728535,
 'transmission': 0.0019567570152718705,
 'ext_col': -0.000151164957787997,
 'int_col': -9.600258240538562e-05,
 'accident': 0.0005223730890825395,
 'clean_title': 2.984194722999156e-06,
 'year_category': -0.00015320010406916218,
 'brand_category': -7.650299164008167e-05,
 'model_category': -0.00011709064545439851,
 'mileage_category': 8.734698476542402e-05,
 'brand_model': 0.00019165046554514564,
 'engine': 0.0034604651716090284}

In [ ]:
FEATS_forw

['accident',
 'average_prices_brand',
 'average_prices_model',
 'average_prices_year',
 'brand',
 'brand_model',
 'brand_model_price',
 'clean_title',
 'engine',
 'fuel_type',
 'milage',
 'mileage_category',
 'model',
 'model_year',
 'relative_mileage',
 'transmission']

# Bagging

In [12]:
FEATS_forw = ['accident', 'average_prices_brand', 'average_prices_model', 'average_prices_year',  'engine', 'ext_col', 'fuel_type', 'int_col', 'milage', 'model', 'model_year', 'transmission']

In [13]:
litle_param = {
    'objective': 'RMSE',
    'iterations': 500,
    'learning_rate': 0.01,
    'depth': 4,
    'l2_leaf_reg': 0.8315956373792702,
    'random_strength': 0.0010708048158020599,
    'bagging_temperature': 6.069384575830549,
    'od_type': 'IncToDec',
    'od_wait': 53,
}

In [14]:
len(FEATS_forw)

12

In [ ]:
models = []

cat_features = [feat for feat in FEATS_forw if feat in cat_cols]
text_features = [feat for feat in FEATS_forw if feat in text_col]
n_models = 200
for i in tqdm(range(n_models)):
  X_train_temp = X_train.sample(frac=0.6,replace=True)
  y_t = y.loc[X_train_temp.index]
  train_pool = Pool(
          X_train_temp[FEATS_forw].values, y_t.values,
          text_features = text_features,
          feature_names = FEATS_forw,
          cat_features = cat_features
          )

  validation_pool = Pool(
              X_test[FEATS_forw].values, y_test.values,
              text_features = text_features,
              feature_names = FEATS_forw,
              cat_features= cat_features
          )
  model = fit_model(train_pool,validation_pool,litle_param)
  models.append(model)


  0%|          | 0/200 [00:00<?, ?it/s]

0:	learn: 0.8381379	test: 0.8416408	best: 0.8416408 (0)	total: 15.3ms	remaining: 7.62s
499:	learn: 0.5118729	test: 0.5185246	best: 0.5185246 (499)	total: 6.3s	remaining: 0us
bestTest = 0.5185246405
bestIteration = 499
0:	learn: 0.8383000	test: 0.8416934	best: 0.8416934 (0)	total: 10.7ms	remaining: 5.32s
499:	learn: 0.5130104	test: 0.5183618	best: 0.5183618 (499)	total: 3.54s	remaining: 0us
bestTest = 0.518361813
bestIteration = 499
0:	learn: 0.8356594	test: 0.8417037	best: 0.8417037 (0)	total: 10.5ms	remaining: 5.25s
499:	learn: 0.5132856	test: 0.5187353	best: 0.5187353 (499)	total: 6.28s	remaining: 0us
bestTest = 0.5187353238
bestIteration = 499
0:	learn: 0.8374332	test: 0.8416976	best: 0.8416976 (0)	total: 23.4ms	remaining: 11.7s
499:	learn: 0.5099727	test: 0.5186390	best: 0.5186390 (499)	total: 3.58s	remaining: 0us
bestTest = 0.5186389564
bestIteration = 499
0:	learn: 0.8414478	test: 0.8415112	best: 0.8415112 (0)	total: 10.8ms	remaining: 5.38s
499:	learn: 0.5143267	test: 0.5183321	b

# Optune for k folds

In [ ]:
FEATS_forw = ['accident', 'average_prices_brand', 'average_prices_model', 'average_prices_year', 'clean_title', 'engine', 'ext_col', 'fuel_type', 'int_col', 'milage', 'model', 'model_year', 'transmission']

In [ ]:
print(len(feats))
print(len(FEATS_forw))

17
13


In [ ]:
from sklearn.metrics import mean_squared_error

def objective(trial):
    # Определение гиперпараметров
    model = CatBoostRegressor(
        objective = trial.suggest_categorical("objective", ["RMSE"]),
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        # bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 50, 60),
        verbose=False,
        task_type="GPU"
    )

    # Обучение модели
    model.fit(train_pool,eval_set=validation_pool, verbose=False)

    # Предсказания
    y_train_pred = model.predict(X_train[FEATS_forw])
    y_val_pred = model.predict(X_test[FEATS_forw])

    # Вычисление ошибок
    train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
    val_rmse = mean_squared_error(y_test, y_val_pred, squared=False)

    # Учет переобучения
    score = 100000000 if val_rmse/train_rmse>1.1 else val_rmse

    return score

In [ ]:
# optuna.logging.set_verbosity(optuna.logging.WARNING)
kf = KFold(n_splits=5,shuffle=True,random_state=42)
cat_features = [feat for feat in FEATS_forw if feat in cat_cols]
text_features = [feat for feat in FEATS_forw if feat in text_col]
k = 0
params = []
for train_index, test_index in kf.split(X):
    k+=1
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    train_pool = Pool(
            X_train[FEATS_forw].values, y_train.values,
            text_features = text_features,
            feature_names = FEATS_forw,
            cat_features = cat_features
            )

    validation_pool = Pool(
                X_test[FEATS_forw].values, y_test.values,
                text_features = text_features,
                feature_names = FEATS_forw,
                cat_features= cat_features
            )
    sampler = TPESampler(seed=1)
    study = optuna.create_study(study_name=f"catboost{k}", direction="minimize", sampler=sampler)
    study.optimize(objective, n_trials=50)
    print("Model is: ",k)
    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial
    print("  Value: ", trial.value)
    print("  Params: ")
    params.append(trial.params)
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    print("-----------------------")
    print()

[I 2024-12-06 11:44:42,371] A new study created in memory with name: catboost1
[I 2024-12-06 11:44:49,856] Trial 0 finished with value: 0.5022140719523588 and parameters: {'objective': 'RMSE', 'iterations': 475, 'learning_rate': 0.027583475549166746, 'depth': 4, 'l2_leaf_reg': 1.0551779964424746e-05, 'random_strength': 2.0931628460945333e-07, 'bagging_temperature': 0.923385947687978, 'od_type': 'Iter', 'od_wait': 54}. Best is trial 0 with value: 0.5022140719523588.
[I 2024-12-06 11:45:12,153] Trial 1 finished with value: 0.5026459948093425 and parameters: {'objective': 'RMSE', 'iterations': 585, 'learning_rate': 0.006892694481137703, 'depth': 8, 'l2_leaf_reg': 1.10795595820296e-06, 'random_strength': 0.7999391045172093, 'bagging_temperature': 0.27387593197926163, 'od_type': 'IncToDec', 'od_wait': 56}. Best is trial 0 with value: 0.5022140719523588.
[I 2024-12-06 11:45:24,040] Trial 2 finished with value: 0.6577151407643269 and parameters: {'objective': 'RMSE', 'iterations': 226, 'learn

KeyboardInterrupt: 

In [ ]:
params = [{
    'objective': 'RMSE',
    'iterations': 7000,
    'learning_rate': 0.04077615905236348,
    'depth': 8,
    'l2_leaf_reg': 0.8315956373792702,
    'random_strength': 0.0010708048158020599,
    'bagging_temperature': 6.069384575830549,
    'od_type': 'IncToDec',
    'od_wait': 53,
},
    {
        'objective': 'RMSE',
    'iterations': 7000,
    'learning_rate': 0.03417611025716251,
    'depth': 8,
    'l2_leaf_reg': 1.6733490133489086,
    'random_strength': 2.8615754410790403e-08,
    'bagging_temperature': 0.06412545920721562,
    'od_type': 'IncToDec',
    'od_wait': 51,
    },
    {
      'objective': 'RMSE',
    'iterations': 7000,
    'learning_rate': 0.08032043510602653,
    'depth': 8,
    'l2_leaf_reg': 8.28798056370749,
    'random_strength': 2.8615754410790403e-08,
    'bagging_temperature': 0.06904013404517156,
    'od_type': 'Iter',
    'od_wait': 53,
    },
    {
        'objective': 'RMSE',
    'iterations': 7000,
    'learning_rate': 0.06133678640678424,
    'depth': 8,
    'l2_leaf_reg': 0.002544841320323785,
    'random_strength': 0.0009541421870155412,
    'bagging_temperature': 8.41644275190244,
    'od_type': 'IncToDec',
    'od_wait': 54,
    },
    {
            'objective': 'RMSE',
    'iterations': 7000,
    'learning_rate': 0.04503275104891686,
    'depth': 8,
    'l2_leaf_reg': 3.770789894183076e-05,
    'random_strength': 0.00014066476962231632,
    'bagging_temperature': 0.6514448802435462,
    'od_type': 'Iter',
    'od_wait': 51,
    }
]

# Train k folds

In [15]:
FEATS_forw = ['accident',
 'average_prices_brand',
 'average_prices_model',
 'average_prices_year',
 'brand',
 'brand_model',
 'brand_model_price',
 'clean_title',
 'engine',
 'fuel_type',
 'milage',
 'mileage_category',
 'model',
 'model_year',
 'relative_mileage',
 'transmission']

FEATS_forw_lgb = ['accident',
 'average_prices_brand',
 'average_prices_model',
 'average_prices_year',
 'brand',
 'brand_model',
 'brand_model_price',
 'clean_title',
 'fuel_type',
 'milage',
 'mileage_category',
 'model',
 'model_year',
 'relative_mileage',
 'transmission']

In [18]:
fold_param = {
    'objective': 'RMSE',
    'iterations': 1000,
}
lgb_param = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 32
}

In [19]:
import lightgbm as lgb
from lightgbm import LGBMRegressor

In [21]:
cat_features = [feat for feat in FEATS_forw if feat in cat_cols]
num_features = [feat for feat in FEATS_forw if feat in numerical_cols]
text_features = [feat for feat in FEATS_forw if feat in text_col]

X_lgb = X.copy()
for i in (cat_features+text_features):
  X_lgb[i] = X_lgb[i].astype("category")

In [23]:
kf = KFold(n_splits=2,shuffle=True,random_state=42,)

k = 0
preds = []
models = []

preds_lgb = []
models_lgb = []
for train_index, test_index in kf.split(X):

    X_train, X_test = X.loc[train_index], X.loc[test_index]
    X_train_lgb, X_test_lgb = X_lgb.loc[train_index], X_lgb.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]


    train_data = lgb.Dataset(X_train_lgb[FEATS_forw_lgb], label=y_train,
                             categorical_feature = cat_features,
                             feature_name= FEATS_forw_lgb
                             )
    test_data = lgb.Dataset(X_test_lgb[FEATS_forw_lgb], label=y_test,
                            categorical_feature = cat_features,
                            reference=train_data,
                            feature_name= FEATS_forw_lgb
                            )
    num_round = 1000
    lgb_model = lgb.train(lgb_param, train_data, num_round, valid_sets=[
                test_data])
    models_lgb.append(lgb_model)
    pred_lgb = lgb_model.predict(X_test_lgb[FEATS_forw_lgb])
    preds_lgb.append(pred_lgb)
    print(mean_squared_error(y_test,pred_lgb,squared=False))

    train_pool = Pool(
            X_train[FEATS_forw].values, y_train.values,
            text_features = text_features,
            feature_names = FEATS_forw,
            cat_features = cat_features
            )

    validation_pool = Pool(
                X_test[FEATS_forw].values, y_test.values,
                text_features = text_features,
                feature_names = FEATS_forw,
                cat_features= cat_features
            )
    model = fit_model(train_pool,validation_pool,fold_param)
    preds.append(model.predict(validation_pool))
    models.append(model)
    k+=1



[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011375 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4574
[LightGBM] [Info] Number of data points in the train set: 94266, number of used features: 15
[LightGBM] [Info] Start training from score 10.290601
0.5067938769776605
Learning rate set to 0.101471
0:	learn: 0.7949489	test: 0.7955027	best: 0.7955027 (0)	total: 80.9ms	remaining: 1m 20s
999:	learn: 0.4691126	test: 0.4936576	best: 0.4936576 (999)	total: 9.86s	remaining: 0us
bestTest = 0.4936575908
bestIteration = 999
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin

In [24]:
o = 0
new_y = []
for x_tr, x_val in kf.split(X):
    new_y.extend(x_val)
new_tar = y[np.array(new_y)]
fin_df = pd.DataFrame()
fin_df["id"] = new_y
fin_df["target"] = new_tar.values
feats = []

for x_tr, x_val in kf.split(X):
    new_df_train = pd.DataFrame()
    new_df_train["id"] = np.array(x_val)
    new_df_train[f"score_{o}"] = preds[o]
    new_df_train[f"score_lgb_{o}"] = preds_lgb[o]
    feats.append(f"score_{o}")
    feats.append(f"score_lgb_{o}")
    o+=1
    fin_df =fin_df.merge(new_df_train,on="id",how="left")

for i in feats:
    fin_df[i].fillna(-1.,inplace=True)

In [36]:
X["id"] = X.index
fin_df2 = fin_df.merge(X[["id"] + FEATS_forw],on="id",how="inner")

X = fin_df2.copy()
y = fin_df2['target']
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=41, train_size=0.8, shuffle=True)

# Optune meta model

In [ ]:
from sklearn.metrics import mean_squared_error

def objective(trial):
    # Определение гиперпараметров
    model = CatBoostRegressor(
        objective = trial.suggest_categorical("objective", ["RMSE"]),
        iterations=trial.suggest_int("iterations", 6000, 7000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        # bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 50, 60),
        verbose=False,
        task_type="GPU"
    )

    # Обучение модели
    model.fit(train_pool,eval_set=validation_pool, verbose=False)

    # Предсказания
    y_train_pred = model.predict(X_train[final_feats])
    y_val_pred = model.predict(X_test[final_feats])

    # Вычисление ошибок
    train_rmse = mean_squared_error(y_train.values, y_train_pred, squared=False)
    val_rmse = mean_squared_error(y_test.values, y_val_pred, squared=False)

    # Учет переобучения
    score = 100000000 if val_rmse/train_rmse>1.1 else val_rmse

    return score

In [ ]:
final_feats = FEATS_forw +feats
cat_features = [feat for feat in final_feats if feat in cat_cols]
text_features = [feat for feat in final_feats if feat in text_col]
train_pool = Pool(
        X_train[final_feats].values, y_train.values,
            text_features = text_features,
            feature_names = final_feats,
            cat_features = cat_features
        )

validation_pool = Pool(
            X_test[final_feats].values, y_test.values,
            text_features = text_features,
            feature_names = final_feats,
            cat_features = cat_features
        )
sampler = TPESampler(seed=1)
study = optuna.create_study(study_name=f"catboost{k}", direction="minimize", sampler=sampler)
study.optimize(objective, n_trials=50)
print("Model is: ",k)
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
print("-----------------------")
print()

[I 2024-12-08 17:25:28,516] A new study created in memory with name: catboost6
[I 2024-12-08 17:26:10,085] Trial 0 finished with value: 0.48985799941956093 and parameters: {'objective': 'RMSE', 'iterations': 6417, 'learning_rate': 0.027583475549166746, 'depth': 4, 'l2_leaf_reg': 1.0551779964424746e-05, 'random_strength': 2.0931628460945333e-07, 'bagging_temperature': 0.923385947687978, 'od_type': 'Iter', 'od_wait': 54}. Best is trial 0 with value: 0.48985799941956093.
[I 2024-12-08 17:28:38,419] Trial 1 finished with value: 0.4872451144829924 and parameters: {'objective': 'RMSE', 'iterations': 6539, 'learning_rate': 0.006892694481137703, 'depth': 8, 'l2_leaf_reg': 1.10795595820296e-06, 'random_strength': 0.7999391045172093, 'bagging_temperature': 0.27387593197926163, 'od_type': 'IncToDec', 'od_wait': 56}. Best is trial 1 with value: 0.4872451144829924.
[I 2024-12-08 17:31:22,362] Trial 2 finished with value: 0.4946103466685633 and parameters: {'objective': 'RMSE', 'iterations': 6140, '

Model is:  6
Number of finished trials:  50
Best trial:
  Value:  0.4864222388080649
  Params: 
    objective: RMSE
    iterations: 6096
    learning_rate: 0.07044966006470568
    depth: 10
    l2_leaf_reg: 67.89625530405097
    random_strength: 0.00013003202081014276
    bagging_temperature: 0.36795217669671937
    od_type: Iter
    od_wait: 58
-----------------------



In [ ]:
trial.params

{'objective': 'RMSE',
 'iterations': 6096,
 'learning_rate': 0.07044966006470568,
 'depth': 10,
 'l2_leaf_reg': 67.89625530405097,
 'random_strength': 0.00013003202081014276,
 'bagging_temperature': 0.36795217669671937,
 'od_type': 'Iter',
 'od_wait': 58}

# Train meta model

In [37]:
param = {'objective': 'RMSE',
 'iterations': 7000,
         'use_best_model': True,
 'depth': 9}
# param = {'objective': 'RMSE',
#  'iterations': 6096,
#  'learning_rate': 0.07044966006470568,
#  'depth': 10,
#  'l2_leaf_reg': 67.89625530405097,
#  'random_strength': 0.00013003202081014276,
#  'bagging_temperature': 0.36795217669671937,
#  'od_type': 'Iter',
#  'od_wait': 58}

In [38]:
final_feats = FEATS_forw +feats
cat_features = [feat for feat in final_feats if feat in cat_cols]
text_features = [feat for feat in final_feats if feat in text_col]
train_pool = Pool(
        X_train[final_feats].values, y_train.values,
            text_features = text_features,
            feature_names = final_feats,
            cat_features = cat_features
        )

validation_pool = Pool(
            X_test[final_feats].values, y_test.values,
            text_features = text_features,
            feature_names = final_feats,
            cat_features = cat_features
        )
model = fit_model(train_pool,validation_pool,param)

Learning rate set to 0.06131
0:	learn: 0.8142278	test: 0.8098210	best: 0.8098210 (0)	total: 133ms	remaining: 15m 30s
1000:	learn: 0.4665413	test: 0.4877198	best: 0.4877176 (995)	total: 30s	remaining: 2m 59s
2000:	learn: 0.4508635	test: 0.4876429	best: 0.4874980 (1563)	total: 55.1s	remaining: 2m 17s
3000:	learn: 0.4373423	test: 0.4880013	best: 0.4874980 (1563)	total: 1m 20s	remaining: 1m 47s
4000:	learn: 0.4249906	test: 0.4887699	best: 0.4874980 (1563)	total: 1m 45s	remaining: 1m 19s
5000:	learn: 0.4136088	test: 0.4898396	best: 0.4874980 (1563)	total: 2m 11s	remaining: 52.6s
6000:	learn: 0.4029771	test: 0.4908283	best: 0.4874980 (1563)	total: 2m 37s	remaining: 26.3s
6999:	learn: 0.3928382	test: 0.4918419	best: 0.4874980 (1563)	total: 3m 3s	remaining: 0us
bestTest = 0.4874980471
bestIteration = 1563
Shrink model to first 1564 iterations.


In [39]:
data = pd.DataFrame({'feature_importance': model.get_feature_importance(train_pool),
              'feature_names': model.feature_names_}).sort_values(by=['feature_importance'],
                                                       ascending=False)

In [40]:
data

,feature_importance,feature_names
10,18.910158,milage
2,13.044812,average_prices_model
13,12.788511,model_year
8,12.541539,engine
3,7.738818,average_prices_year
17,4.804672,score_lgb_0
19,4.521129,score_lgb_1
15,4.436319,transmission
14,3.975370,relative_mileage
18,3.858336,score_1


In [41]:
y_pred = model.predict(validation_pool)
mean_squared_error(y_test, y_pred)

0.2376543147768487

In [42]:
y_pred = model.predict(X_test[final_feats])
mean_squared_error(y_test, y_pred)

0.2376543147768487

In [44]:
mean_squared_error(np.exp(y_test), np.exp(y_pred),squared=False)

72017.33063790761

In [ ]:
mean_squared_error(np.exp(y_test), np.exp(y_pred))

4717543039.900205

# Predict

In [45]:
feats

['score_0', 'score_lgb_0', 'score_1', 'score_lgb_1']

In [47]:
test_new_lgb = test_new.copy()
for i in cat_features:
  test_new_lgb[i] = test_new_lgb[i].astype("category")

In [48]:
# # stacking
cat_features = [feat for feat in FEATS_forw if feat in cat_cols]
text_features = [feat for feat in FEATS_forw if feat in text_col]

test_pool = Pool(
    test_new[FEATS_forw],
    text_features = text_features,
    cat_features= cat_features,
    feature_names = FEATS_forw
)
y_preds = []
for m in models:
  y_pred = m.predict(test_pool)
  y_preds.append(y_pred)

y_preds_lgb = []
for m in models_lgb:
  pred_lgb = m.predict(test_new_lgb[FEATS_forw_lgb])
  y_preds_lgb.append(pred_lgb)

test_df = pd.DataFrame()

for i in range(len(models)):
  test_df[f"score_{i}"] = y_preds[i]

for i in range(len(models_lgb)):
  test_df[f"score_lgb_{i}"] = y_preds_lgb[i]
test_df["ids"] = range(len(test_df))

fin_test = test_new.merge(test_df,on="ids",how="inner")

fin_preds = model.predict(fin_test[final_feats])

In [ ]:
# bagging
# cat_features = [feat for feat in FEATS_forw if feat in cat_cols]
# text_features = [feat for feat in FEATS_forw if feat in text_col]

# test_pool = Pool(
#     test_new[FEATS_forw],
#     text_features = text_features,
#     cat_features= cat_features,
#     feature_names = FEATS_forw
# )
# y_preds = []
# for m in tqdm(models):
#   y_pred = m.predict(test_pool)
#   y_preds.append(y_pred)
# fin_preds = np.mean(y_preds,axis=0)

In [49]:
submission = pd.read_csv("sample_submission.csv")
print(submission.shape)
submission['price'] = np.exp(fin_preds)
submission

(125690, 2)


,id,price
0,188533,16172.805475
1,188534,56050.862196
2,188535,44860.799139
3,188536,39914.544955
4,188537,31185.293950
...,...,...
125685,314218,38748.803117
125686,314219,39954.153541
125687,314220,18094.037704
125688,314221,12340.188837


In [ ]:
submission = pd.read_csv("sample_submission.csv")
print(submission.shape)
submission['price'] = np.exp(fin_preds)
submission

(125690, 2)


,id,price
0,188533,29468.774390
1,188534,29480.136083
2,188535,29433.351665
3,188536,29495.789790
4,188537,29479.888559
...,...,...
125685,314218,29453.788941
125686,314219,29430.290880
125687,314220,29468.774390
125688,314221,29383.968225


In [50]:
submission.to_csv("submission_new_no_exp25.csv", index = False)